In [1]:
# coding: utf-8

# In[1]:




# In[15]:


import time
import nltk
import sys
import operator

In [2]:
# In[2]:


# In[16]:


stop_file = open("Stop_words.txt", "r")
content = stop_file.read()
sum_text=0
stop_words={}

In [3]:
# In[3]:


# In[17]:


import re
reg = re.compile("\"|,| ")
import nltk
stemmer = nltk.stem.SnowballStemmer('english')
content = re.split(reg, content)
sum_title=0
for word in content :
    if word :
        stop_words[word] = True

In [6]:
# In[4]:


# In[ ]:


# folderPath=sys.argv[1]
folderPath="IndexSize"

In [ ]:
# In[5]:


# In[18]:


titleTags = open(folderPath+"/title_tags.txt", "r")
resutltantListFieldQuery=dict()


# In[6]:


# In[19]:

sys.argv[1]
start1 = time.time()



import pickle
titlePosition = pickle.load(open(folderPath+"/titlePositions.pickle", "rb"))
wordPosition = pickle.load(open(folderPath+"/wordPositions.pickle", "rb"))
end1 = time.time()

In [8]:
print("Pickle load in - " + str(end1 - start1) + "s")

Pickle load in - 1373.8309071063995s


In [ ]:
# In[7]:


# In[20]:


def findTitle(documentList):
#     print(documentList)
    titleList=[]
    if "," in documentList:
        list1=documentList.split(",")
        for doc in list1:
            docNo,count=doc.split(":")
            position=titlePosition[int(docNo)-1]
            titleTags.seek(position)
            title = titleTags.readline()[: -1]
#             print(title)
            titleList.append(title)
    else:
        docNo,count=documentList.split(":")
        position=titlePosition[int(docNo)-1]
        titleTags.seek(position)
        title = titleTags.readline()[: -1]
        titleList.append(title)
        
    return titleList

In [9]:
# In[8]:


# In[21]:


def findDocumentList(position,fp):
    fp.seek(position)
    s=fp.readline()[: -1]
    return s

In [10]:
# In[22]:


# In[9]:


file = folderPath+"/titlePosting.txt"
fp_title = open(file, "r")

file = folderPath+"/infoBoxPosting.txt"
fp_infoBox = open(file, "r")

file = folderPath+"/categoryPosting.txt"
fp_category = open(file, "r")

file = folderPath+"/textPosting.txt"
fp_text = open(file, "r")

In [66]:
# In[10]:


# In[23]:


def searchPosition(word,type1):
    return wordPosition[word][type1]
#     documentList=[]
#     for key,document in wordPosition:
#         if key==type1:
#             documentList=document
#     return documentList


# In[24]:

In [67]:
# In[11]:


def intersection(lst1, lst2): 
    tup1 = map(tuple, lst1) 
    tup2 = map(tuple, lst2)  
    return list(map(list, set(tup1).intersection(tup2))) 

In [68]:
# In[25]:


# In[12]:


def updateResultantList(final_doc):
    global resutltantListFieldQuery
    # print(type(resutltantListFieldQuery))
    for i in final_doc:
        docNo,tfIdfScore=i.split(":")
#         print(tfIdfScore)
        if docNo not in resutltantListFieldQuery:
            resutltantListFieldQuery[int(docNo)]=float(tfIdfScore)
        else:
            resutltantListFieldQuery[int(docNo)]+=float(tfIdfScore)


# In[13]:

In [69]:
def searchWordOfNonFieldQuery(word):
    global resutltantListFieldQuery
    positions=wordPosition[word]
#     print(positions)
    result=[]
    for tag in positions:
        position=positions[tag]
        if tag=="t":
            documentList=findDocumentList(position,fp_title)
            # print(documentList)
            final_doc=[]
            if "," in documentList:
                final_doc=documentList.split(",")
            else:
                final_doc.append(documentList)
            updateResultantList(final_doc)
            
        if tag=="d":
            documentList=findDocumentList(position,fp_text)
            final_doc=[]
            if "," in documentList:
                final_doc=documentList.split(",")
            else:
                final_doc.append(documentList)
            updateResultantList(final_doc)
            
        if tag=="c":
            documentList=findDocumentList(position,fp_category)
            final_doc=[]
            if "," in documentList:
                final_doc=documentList.split(",")
            else:
                final_doc.append(documentList)
            updateResultantList(final_doc)
        
        if tag=="i":
            documentList=findDocumentList(position,fp_infoBox)
            final_doc=[]
            if "," in documentList:
                final_doc=documentList.split(",")
            else:
                final_doc.append(documentList)
            
            updateResultantList(final_doc)


# In[14]:

In [79]:
resultantList=dict()
def search():
    while True:
        finalResult=[]
    #     noOfQueries=len(queries)
    #     print(noOfQueries)
        global resutltantListFieldQuery
        global resultantList
    #     for data in range(0,noOfQueries):
        query = input()
        start = time.time()
        print(query)
        result=[]
    #     query=queries[data]
        flag=1
        query=query.lower()
        query=query.strip()
        if query=="exit":
            break
        resultantList=dict()
        resutltantListFieldQuery={}
        if ":" in query:
            flag=0
            queryTags=[]
            # if " " in query:
            #     queryTags=query.split(",")
            # else:
            #     queryTags.append(query)
            queryTags=query.split(" ")
            for tag in queryTags:
                # print(tag)
                tagName,words=tag.split(":")
                count1=0
                # print(tagName)
                words=words.split()
                for word in words:
                    # print(tagName)
                    tagName=tagName.strip()
                    # word=word.strip()
                    word=stemmer.stem(word)
                    if word not in wordPosition:
                        continue
                    if tagName=="title":
                        tagName="t"
                        if tagName not in wordPosition[word]:
                            continue
                        # print(tag)
                        position=searchPosition(word,"t")
    #                     print(word,position)
    #                     print("ARe u kiddign")
                        if position==0:
                            continue
                        documentList=findDocumentList(position,fp_title)
    #                         titleList_t=findTitle(documentList)
    #                         resultantList.append(titleList_t)

                    if tagName=="body" or tagName == "ref" or tagName=="ext":
                        tagName="d"
                        if tagName not in wordPosition[word]:
                            continue
                        position=searchPosition(word,"d")
                        if position=="":
                            continue
                        if tagName not in wordPosition[word]:
                            continue
    #                     print("ARe u kiddignddddddddddddddddddddddddddddd")
                        documentList=findDocumentList(position,fp_text)
    #                         titleList_d=findTitle(documentList)
    #                         resultantList.append(titleList_d)
                    if tagName=="infobox":
                        tagName="i"
                        if tagName not in wordPosition[word]:
                            continue
                        position=searchPosition(word,"i")
                        if position=="":
                            continue
    #                     print("ARe u kiddiiiiiiiiiiiiiiiiiiiii")
                        documentList=findDocumentList(position,fp_infoBox)
    #                         titleList_i=findTitle(documentList)
    #                         resultantList.append(titleList_i)
                    if tagName=="category":
                        tagName="c"
                        if tagName not in wordPosition[word]:
                            continue
                        position=searchPosition(word,"c")
                        if position=="":
                            continue
    #                     print("ARe u kiddcccccccccccccccccccccccccccc")
                        documentList=findDocumentList(position,fp_category)
    #                         titleList_c=findTitle(documentList)
    #                         resultantList.append(titleList_c)
                    final_doc=[]
                    if "," in documentList:
                        final_doc=documentList.split(",")
                    else:
                        final_doc.append(documentList)

                    for i in final_doc:
                        docNo,tfIdfScore=i.split(":")
                        # print(resultantList)
                        # print(type(resultantList))
                        if docNo not in resultantList:
                            resultantList[int(docNo)]=float(tfIdfScore)
                        else:
                            resultantList[int(docNo)]+=float(tfIdfScore)

            resultantList = sorted(resultantList.items(), key = operator.itemgetter(1),reverse=True)
            for docs in resultantList:
                titleTags.seek(titlePosition[int(docs[0])-1])
                result.append(titleTags.readline()[: -1])

        else:
            words=query.split()
            # print("words ",words)
            for word in words:
                word=stemmer.stem(word)
                if word not in wordPosition:
                    continue
                searchWordOfNonFieldQuery(word)
            resutltantListFieldQuery = sorted(resutltantListFieldQuery.items(), key = operator.itemgetter(1),reverse=True)
    #             print(resutltantListFieldQuery)
            for docs in resutltantListFieldQuery:
    #                 print(docs[0])
                titleTags.seek(titlePosition[int(docs[0])-1])
                result.append(titleTags.readline()[: -1])


        if len(result)>10:
            result=result[0:10]

        end = time.time()
        print(result)
        print(" Results in - " + str(end - start) + "s")
#         return result
#     return finalResult

In [71]:
        
    #     print("lengthhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh    ",len(resultantList))

        # print(resultantList)
        # resutltantListFieldQuery
#         if len(resultantList)==0 and flag==0:
#             # print("ddd")
#             resultantList.append("No result found")
#             result.append(resultantList[0])

#         elif len(resutltantListFieldQuery)==0 and flag==1:
#             # print("dddd")
#             resutltantListFieldQuery.append("No result found")
#             result.append(resutltantListFieldQuery[0])
#         elif flag==0:
#             ans=[]
#             if len(resultantList)==1:
#                 # print("sssss")
#                 if len(resultantList[0]) > 10:
#                     # print(len(resultantList[0]))
#                     resultantList[0]=resultantList[0][:10]
#                 result.append(resultantList[0])
#                 # for i in resultantList[0]:
#                 #     print(i)
#                 continue

#             n=len(resultantList)
#             ans=resultantList[0]
#             for i in range(1,n):
#                 setA=set(ans)
#     #             print(setA)
#                 setB=set(resultantList[i])
#                 ans=list(setA & setB )
#             # print("ssssss ",len(ans))
#             if len(ans) > 10:
#                 ans=ans[:10]
#             if len(ans)==0:
#                 ans.append("No result found")
#             result.append(ans)
#             # for i in ans:
#             #     print(i)
#         else:
#             ans=[]
#     #         print(resutltantListFieldQuery)
#             if len(resutltantListFieldQuery)==1:
#                 # print("ssssss")
#                 # print(len(resutltantListFieldQuery[0]))
#                 if len(resutltantListFieldQuery[0]) >10:
#                     resutltantListFieldQuery[0]=resutltantListFieldQuery[0][:10]
#                 result.append(resutltantListFieldQuery[0])
#                 # for i in resutltantListFieldQuery[0]:
#                 #     print(i)
#                 continue

#             n=len(resutltantListFieldQuery)
#             # print(n)
#             ans=resutltantListFieldQuery[0]
#             # print(len(ans))
#     #         t = list(set(q) & set(w))
#             for i in range(1,n):
#     #             print("inside")
#                 setA=set(ans)
#     #             print(setA)
#                 setB=set(resutltantListFieldQuery[i])
#                 # print(len(setB))
#                 ans=list(setA & setB )
#                 # print(len(ans))
#     #             print("dd ",ans)
#             # print(len(ans))
#             # print("sss ",len(ans))
#             if len(ans) > 10:
#                 ans=ans[:10]

#             if len(ans)==0:
#                 ans.append("No result found")
#             result.append(ans)
            # for i in ans:
            #     print(i)
   

In [72]:
def read_file(testfile):
    with open(testfile, 'r') as file:
        queries = file.readlines()
    return queries

In [73]:
# In[16]:


# In[28]:


# In[17]:


def write_file(outputs, path_to_output):
    '''outputs should be a list of lists.
        len(outputs) = number of queries
        Each element in outputs should be a list of titles corresponding to a particular query.'''
    with open(path_to_output, 'w') as file:
        # print(len(outputs))
        # print(len(outputs))
        if len(outputs) > 0:
            
            for output in outputs:
                for line in output:
                    file.write(line.strip() + '\n')
                file.write('\n')

In [74]:
# In[29]:


# In[18]:


# def main():
#     print("dddddddd")
#     path_to_index = sys.argv[1]
# #     path_to_index = "IndexSize"
#     testfile = sys.argv[2]
#     path_to_output = sys.argv[3]

#     queries = read_file(testfile)
#     print(queries)
# #     queries=[]
# #     queries.append("Gandhi india")
#     start = time.time()
#     outputs = search(path_to_index, queries)
#     # print(outputs)
#     # print("len ", len(outputs),outputs)
#     end = time.time()
#     print("Results in - " + str(end - start) + "s")

#     write_file(outputs, path_to_output)


# if __name__ == '__main__':
#     main()  

In [80]:
path_to_index = "IndexSize"
#     path_to_index = "IndexSize"
testfile = "queryfile"
# path_to_output = "result"

# queries = read_file(testfile)
# print(queries)
#     queries=[]
#     queries.append("Gandhi india")
print("Enter the query")
# queries=input()
search()
# print(outputs)

# print(outputs)
# print("len ", len(outputs),outputs)


Enter the query
gandhi
gandhi
['Mahatma Gandhi', 'Indira Gandhi', 'Gandhi family', 'Salt March', 'Gandhism', 'Wikipedia:WikiProject Spam/LinkReports/faculty.georgetown.edu', 'Vallabhbhai Patel', 'Sonia Gandhi', 'Nehru–Gandhi family', 'Rahul Gandhi']
 Results in - 0.008433341979980469s
title:gandhi
title:gandhi
['Gandhi Mate, Gandhi', "File:Indira Gandhi's study room, Indira Gandhi Memorial Museum, New Delhi.JPG", 'Statue of Mahatma Gandhi, Gandhi Maidan', 'File:Major HPS Ahluwalia with rajiv Gandhi and Sonia Gandhi.jpg', 'File:Gandhi Samadhi memorial to Mahatma Gandhi.jpg', 'Mahatma Gandhi bus station, Hyderabad', 'Pravinchandra Varjivan Gandhi', 'Indira Gandhi Institute of Technology', "Mahatma Gandhi Mission's College of Engineering and Technology", 'Rajiv Gandhi International Cricket Stadium, Uppal']
 Results in - 0.002448558807373047s
exit
exit
